# Animated Scatterplot

#### Initial setup

In [1]:
import pandas as pd
import plotly.express as px
#import matplotlib.pyplot as plt
#import seaborn as sns
#import imageio
#plt.style.use('ggplot')

#### Import data:

In [2]:
life = pd.read_excel('../data/gapminder_lifeexpectancy.xlsx', index_col=0)
fert = pd.read_csv('../data/gapminder_total_fertility.csv', index_col=0)
popu = pd.read_excel("../data/gapminder_population.xlsx", index_col=0)
cont = pd.read_csv("../data/continents.csv", sep=";", index_col=1)

#### Check shape:

In [3]:
print(life.shape)
print(fert.shape)
print(popu.shape)
print(cont.shape)

(260, 217)
(260, 216)
(275, 81)
(194, 1)


#### Check column dtype:

In [4]:
fert.columns = fert.columns.astype(int)

print(life.columns.dtype)
print(fert.columns.dtype)
print(popu.columns.dtype)
# cont doesn't have a "year" feature -> no check needed

int64
int64
int64


#### Correct index names:

In [5]:
life.index.name="country"
fert.index.name="country"
popu.index.name="country"

#### Reset index for melting:

In [6]:
fert = fert.reset_index()
life = life.reset_index()
popu = popu.reset_index()
cont = cont.reset_index()

#### Like ice in the sunshine:

In [7]:
# cont already has right format for merging!

fert = fert.melt(id_vars='country', var_name='year', value_name='fertility_rate')
life = life.melt(id_vars='country', var_name='year', value_name='life_expectancy')
popu = popu.melt(id_vars='country', var_name='year', value_name='population')

#### Merge all DF's and drop NaN Rows

In [8]:
merged_df = fert.merge(popu)
merged_df = merged_df.merge(life)
merged_df = merged_df.merge(cont)
merged_df = merged_df.dropna()
merged_df.sample(10)

,country,year,fertility_rate,population,life_expectancy,continent
7977,Malta,1974,2.03,306291.0,72.78,Europe
1253,Belarus,1973,2.22,9245514.0,72.29,Europe
13751,Vanuatu,1997,4.61,175004.0,61.20,Australia and Oceania
5738,Iceland,2003,2.02,289824.0,81.30,Europe
10006,Paraguay,1978,5.20,3012833.0,71.28,South America
3043,Costa Rica,1981,3.58,2454127.0,73.73,North America
7880,Mali,1958,6.66,5147344.0,29.23,Africa
2273,Cameroon,1850,5.54,2033578.0,28.75,Africa
3747,Ecuador,1956,6.72,4066240.0,51.23,South America
11498,Solomon Islands,2012,4.10,549162.0,63.30,Australia and Oceania


#### Determine minmax values for plotaxis and marker size norm:

In [9]:
plot_ymax = merged_df["fertility_rate"].max()
plot_ymin = merged_df["fertility_rate"].min()
plot_xmax = merged_df["life_expectancy"].max()
plot_xmin = merged_df["life_expectancy"].min()
minsize = merged_df["population"].min()
maxsize = merged_df["population"].max()

In [10]:
print(plot_xmin, plot_xmax, plot_ymin, plot_ymax, minsize, maxsize)

4.0 83.3 1.13 9.22 2128.0 1376048943.0


#### Make animated scatterplot with plotly and save as html

In [11]:
fig = px.scatter(merged_df.query("year>=1960"),
                 #title="World Data 1960 to 2015",
                 x="life_expectancy",
                 y="fertility_rate",
                 color="continent",
                 size='population',
                 size_max=45,
                 animation_frame="year",
                 #animation_group="continent",
                 range_x=[0,90],
                 range_y=[0,10],
                 #marginal_x="histogram",
                 #marginal_y="histogram",
                 hover_data=['country'],
                 #text="country",
                 labels={
                     "life_expectancy": "Life Expectancy (years)",
                     "fertility_rate": "Fertility Rate per Person",
                     "continent": "Continent",
                     "year": "Year",
                     "population": "Population",
                     "country": "Country"
                 },
                 
                 template="plotly_dark",
                 #width=1400,
                 #height=800
                 )

fig.update_layout(
    title={
        'text': "Gapminder Data from 1960 to 2015",
        "font": dict(size=20),
        #'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'bottom'})


fig.write_html("Gapminder_plotly_animation.html")
#fig.write_image("Gapminder_plotly.svg")
fig.show()